# Zindi - Challenge

https://zindi.africa/competitions/predict-the-global-spread-of-covid-19

April 9, 2020

In [109]:
import pandas as pd
import seaborn as sns
import matplotlib as plt 
import requests
import lxml.html as lh

Only the folder Global_variable_per-country was used.

## Import Data

In [110]:
#Read train 
deaths = pd.read_csv('data/deaths.csv')
cases = pd.read_csv('data/cases.csv')
deaths.drop(['Province/State'], axis=True)
deaths.shape

(274, 84)

In [111]:
deaths.head(2)

,Province/State,Country/Region,Territory,Population,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20
0,NaN,Afghanistan,Afghanistan,26023100.0,33.0000,65.0000,0,0,0,0,...,4,4,4,6,6,7,7,11,14,14
1,NaN,Albania,Albania,2895947.0,41.1533,20.1683,0,0,0,0,...,11,15,15,16,17,20,20,21,22,22


Compare the 1st 3 columns.

In [112]:
#no missing values
sum(deaths['Country/Region'].isnull() == False)

274

We will use 'Country/Region'. 


In [113]:
country = deaths['Country/Region']

In [114]:
## Fill in NaN values for Population
index = deaths[deaths.Population.isnull()].index
# updated in 2017
deaths.at[index[0], 'Population'] = 76965 
deaths.at[index[1], 'Population'] = 395361
deaths.at[index[2], 'Population'] = 11.05e+06
deaths.at[index[3], 'Population'] = 428697
deaths.at[index[4], 'Population'] = 546388 
deaths.at[index[5], 'Population'] = 4.659e+06
deaths.at[index[6], 'Population'] = 5.261e+06
deaths.at[index[7], 'Population'] = 81.34e+06
deaths.at[index[8], 'Population'] = 24.29e+06 # Cote d'Ivoire
deaths.at[index[10], 'Population'] = 10.77e+06
deaths.at[index[11], 'Population'] = 1.367e+06
deaths.at[index[12], 'Population'] = 2.101e+06
deaths.at[index[13], 'Population'] = 81.16e+06
deaths.at[index[14], 'Population'] = 51.47e+06
deaths.at[index[15], 'Population'] = 3.55e+06
deaths.at[index[17], 'Population'] = 105264
deaths.at[index[18], 'Population'] = 161014
deaths.at[index[19], 'Population'] = 41109 
deaths.at[index[21], 'Population'] = 21.48e+06
deaths.at[index[23], 'Population'] = 104.9e+06
deaths.at[index[24], 'Population'] = 144.5e+06
deaths.at[index[25], 'Population'] = 7.022e+06
deaths.at[index[26], 'Population'] = 12.58e+06
deaths.at[index[28], 'Population'] = 57.31e+06
deaths.at[index[29], 'Population'] = 9.4e+06
deaths.at[index[30], 'Population'] = 65441
deaths.at[index[31], 'Population'] = 61559
deaths.at[index[32], 'Population'] = 170499
deaths.at[index[33], 'Population'] = 34571
deaths.at[index[34], 'Population'] = 84287
deaths.at[index[35], 'Population'] = 5900 #2008
deaths.at[index[38], 'Population'] = 31.98e+06
deaths.at[index[39], 'Population'] = 95.54e+06
deaths.at[index[40], 'Population'] = 18.27e+06
deaths.at[index[41], 'Population'] = 1.296e+06
deaths.at[index[42], 'Population'] = 6.858e+06
deaths.at[index[43], 'Population'] = 15094
deaths.at[index[44], 'Population'] = 31196
deaths.at[index[45], 'Population'] = 35446
deaths.at[index[46], 'Population'] = 25157
deaths.at[index[47], 'Population'] = 2840
deaths.at[index[48], 'Population'] = 204327
deaths.at[index[49], 'Population'] = 813912
deaths.at[index[50], 'Population'] = 53127
deaths.at[index[51], 'Population'] = 105544
deaths.at[index[52], 'Population'] = 53.37e+06
deaths.at[index[53], 'Population'] = 51.47e+06
deaths.at[index[54], 'Population'] = 40.53e+06

#2018 
deaths.at[index[27], 'Population'] = 23.78e+06
deaths.at[index[37], 'Population'] = 327.2e+06
#2019
deaths.at[index[9], 'Population'] = 10.65e+06
deaths.at[index[20], 'Population'] = 17.28e+06
deaths.at[index[22], 'Population'] = 2.077e+06
deaths.at[index[36], 'Population'] = 6.65e+06

#2020
deaths.at[index[16], 'Population'] = 631219





No more NaN values for population. 

Add a code country column.

In [115]:
def web_scraping_data(link):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(link)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')

    tr_elements = doc.xpath('//tr')
    col=[]
    i=0
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        #print ('%d:"%s"'%(i,name))
        col.append((name,[]))

    for j in range(1,len(tr_elements)):
        T=tr_elements[j]
        i=0

        for t in T.iterchildren():
            data=t.text_content() 
            if i>0:
                try:
                    data=int(data)
                except:
                    pass

            col[i][1].append(data)
            i+=1

    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    
    return df

In [118]:
code_country = web_scraping_data("https://www.iban.com/country-codes")
df = deaths.merge(code_country, how='left', left_on='Country/Region', right_on='Country')

In [119]:
## Fill in NaN values for alpha 3 code 
index = df[df['Alpha-3 code'].isnull()].index
variable = 'Alpha-3 code'
df.at[index[0], variable] = 'BHS'
df.at[index[1], variable] = 'BOL'
df.at[index[2], variable] = 'BRN'
df.at[index[3], variable] = 'CAF'
df.at[index[4], variable] = 'COG'
df.at[index[5], variable] = 'COD'
df.at[index[6], variable] = 'CIV'
df.at[index[7], variable] = 'DOM'
df.at[index[8], variable] = 'GMB'
df.at[index[9],variable] = 'IRN'
df.at[index[10],variable] = 'PRK'
df.at[index[11], variable] = 'MDA'
df.at[index[12], variable] = 'ABW'
df.at[index[13], variable] = 'CUW'
df.at[index[14],variable] = 'SXM'
df.at[index[15],variable] = 'NLD'
df.at[index[16],variable] = 'NER'
df.at[index[17], variable] = 'MKD'
df.at[index[18], variable] = 'PHL'
df.at[index[19], variable] = 'RUS'
df.at[index[20], variable] = 'SSD'
df.at[index[21], variable] = 'TWN'
df.at[index[22],variable] = 'TZA'
df.at[index[23],variable] = 'ARE'
df.at[index[24],variable] = 'BMU'
df.at[index[25],variable] = 'CYM'
df.at[index[26],variable] = 'GBR' #by default
df.at[index[27],variable] = 'GIB'
df.at[index[28],variable] = 'IMN'
df.at[index[29],variable] = 'MSR'
df.at[index[30],variable] = 'GBR'
df.at[index[31],variable] = 'USA'
df.at[index[32], variable] = 'VEN'
df.at[index[33],variable] = 'VNM'
df.at[index[34],variable] = 'SYR'
df.at[index[35], variable] = 'LAO'
df.at[index[36], variable] = 'AIA'
df.at[index[37], variable] = 'VGB'
df.at[index[38], variable] = 'TCA'
df.at[index[39],variable] = 'BES'
df.at[index[40], variable] = 'FLK'
df.at[index[41], variable] = 'KOR'


We will use alpha 3 code to join with other tables.

## Add variables


### Temperature 

Source 
- https://datacatalog.worldbank.org/dataset/climate-change-knowledge-portal-historical-data

In [120]:
temp = pd.read_csv('variable/Global_variable_per-country/temp.csv')

In [121]:
temp.head()

,ISO_3DIGIT,Jan_Temp,Feb_temp,Mar_temp,Apr_Temp,May_temp,Jun_Temp,July_Temp,Aug_Temp,Sept_temp,Oct_temp,Nov_Temp,Dec_temp,Annual_temp
0,AFG,0.07,2.11,7.60,13.37,18.22,23.20,25.26,23.77,19.03,12.99,7.00,2.43,12.92
1,AGO,22.58,22.68,22.78,22.35,20.74,18.37,17.95,19.90,22.19,23.18,22.79,22.61,21.51
2,ALB,2.02,3.22,6.04,9.92,14.44,17.93,20.54,20.48,17.16,12.27,7.58,3.65,11.27
3,ARE,18.43,19.43,22.61,26.58,30.62,32.46,33.80,33.55,31.74,28.34,24.06,20.28,26.83
4,ARG,20.80,19.90,17.51,14.05,10.65,7.66,7.42,9.02,11.53,14.67,17.54,19.83,14.22


In [122]:
# temperature = pd.merge(left=temp, right=df, left_on='ISO_3DIGIT', right_on='Alpha-3 code')

df = df.merge(temp, how='left', left_on='Alpha-3 code', right_on='ISO_3DIGIT')
df.drop(['Numeric','ISO_3DIGIT'], axis =1, inplace=True)

In [123]:
sum(df.Annual_temp.isnull())

38

### Other epidemy 

SARS-CoV

China Canada, Hong Kong Special Administrative Region of China, Chinese Taipei, Singapore, and Hanoi in Viet Nam.

Ebola 

The Democratic Republic of the Congo
Gabon
Guinea
Italy
Ivory Coast
Liberia
Mali
Nigeria
Philippines
Russia
Senegal
Sierra Leone
South Africa
South Sudan
Spain
Uganda
United Kingdom
United States
What causes Ebol

### Mers

https://www.kaggle.com/imdevskp/mers-outbreak-dataset-20122019

In [124]:
#nb of case in each country for MERS
mers = pd.read_csv('variable/Global_variable_per-country/mers/country_count_latest.csv')
# mers = pd.read_csv('variable/mers/weekly_clean.csv') # to follow week per week 
mers.shape

(27, 2)

In [125]:
df = df.merge(mers, how='left', left_on='Territory', right_on='Country')
df['Mers-confirmed'] = df.Confirmed 
df.drop(['Confirmed'], axis=1, inplace=True)

In [126]:
df["Mers-confirmed"] = df["Mers-confirmed"].fillna(0)

### Sars 
https://www.kaggle.com/imdevskp/sars-outbreak-2003-complete-dataset

In [127]:
sars = pd.read_csv('variable/Global_variable_per-country/sars/sars_2003.csv')
c = list(sars.Country.unique())
death_nb = []
for country in c:
    v = list(sars[sars.Country == country]['Number of deaths'])[len(sars[sars.Country ==country]['Number of deaths'])-1]
    death_nb.append(v)

In [128]:
sars = pd.DataFrame({'Country':c, 'death_nb_sars': death_nb})
df = df.merge(sars, how='left', left_on='Territory', right_on='Country')
df.drop(['Country_y',  'Country'], axis=1, inplace=True)
df["death_nb_sars"] = df["death_nb_sars"].fillna(0)

### Age / density / wealth /sex 

See relevant indicators

http://datatopics.worldbank.org/universal-health-coverage/coronavirus/



https://datacatalog.worldbank.org/search?search_api_views_fulltext_op=AND&f%5B0%5D=field_collection_field%3A2026&sort_by=field_wbddh_modified_date

https://data.worldbank.org/indicator/en.pop.dnst

In [129]:
def wordbankdata(variable, tab):
    data = pd.read_csv('variable/Global_variable_per-country/'+variable+'.csv')
    data = data[['Country Code', '2018']]
    tab = df.merge(data, how='left', left_on='Alpha-3 code', right_on='Country Code')
    tab[variable] = tab['2018']
    tab.drop(['2018', 'Country Code'], axis=True, inplace=True)
    return(tab)

variable = '%+65'

In [130]:
df = wordbankdata(variable, df)

In [131]:
variable = '15-64pop'
df = wordbankdata(variable, df)
df = wordbankdata('density', df)
df = wordbankdata( 'sex', df)

In [132]:
## not so many NaN cool !
print(sum(df['%+65'].isnull()))
print(sum(df['15-64pop'].isnull()))
print(sum(df['density'].isnull()))
print(sum(df['sex'].isnull()))

23
23
12
23


In [133]:
rich = pd.read_csv('variable/Global_variable_per-country/income.csv')
rich = rich[['Country Code', 'IncomeGroup']]
df = df.merge(rich, how='left', left_on='Alpha-3 code', right_on='Country Code')
df.drop(['Country Code'], axis=1, inplace=True)

In [134]:
print(sum(df['IncomeGroup'].isnull()))

5


## Tourism

In [135]:
tourism = pd.read_csv('variable/Global_variable_per-country/tourism.csv')

tourism.count() / len(tourism)
Lets take 2011.

In [136]:
tourism = tourism[['2011','Country Code']]
df = df.merge(tourism, how='left', left_on='Alpha-3 code', right_on='Country Code')
df.drop(['Country Code'], axis=1, inplace=True)

In [137]:
df['tourism'] = df['2011']
df.drop(['2011'], axis=1, inplace=True)

## Economical factors

https://www.kaggle.com/lewisduncan93/the-economic-freedom-index

In [138]:
econ = pd.read_csv('variable/Global_variable_per-country/economic_freedom_index2019_data.csv',encoding='latin-1')

In [139]:
econ.drop(['Region Rank','Tax Burden % of GDP','Trade Freedom','Financial Freedom','Property Rights','FDI Inflow (Millions)','Judical Effectiveness','Business Freedom','Tax Burden','Labor Freedom','Population (Millions)','Monetary Freedom'], axis=1, inplace=True)

In [140]:
df = df.merge(econ, how='left', left_on='Country/Region', right_on='Country')

In [141]:
df.drop(['Country'], axis=1, inplace=True)

## Immunization factor

https://www.kaggle.com/lsind18/who-immunization-coverage

BCG.csv: the percentage of 1-year-olds who have received one dose of bacille Calmette-Guérin (BCG) vaccine in a given year.

DTP3.csv: the percentage of 1-year-olds who have received three doses of the combined diphtheria, tetanus toxoid and pertussis vaccine in a given year.

HepB3.csv: the percentage of 1-year-olds who have received three doses of hepatitis B vaccine in a given year.

Hib3.csv: the percentage of 1-year-olds who have received three doses of Haemophilus influenzae type B vaccine in a given year.

MCV1.csv: the percentage of children under 1 year of age who have received at least one dose of measles-containing vaccine in a given year. For countries recommending the first dose of measles vaccine in children over 12 months of age, the indicator is calculated as the proportion of children less than 12-23 months of age receiving one dose of measles-containing vaccine.

MCV2.csv: the percentage of children who have received two doses of measles containing vaccine (MCV2) in a given year, according to the nationally recommended schedule.

PAB.csv: the proportion of neonates in a given year that can be considered as having been protected against tetanus as a result of maternal immunization.

PCV3.csv: the percentage of 1-year-olds who have received three doses of pneumococcal conjugate vaccine (PCV3) in a given year.

Pol3.csv: the percentage of 1-year-olds who have received three doses of polio vaccine in a given year.

ROTAC.csv: the percentage of surviving infants who received the final recommended dose of rotavirus vaccine, which can be either the 2nd or the 3rd dose depending on the vaccine in a given year.


who-immunization-coverage folder

We will take the year 2018 (most recent and less NaN values).

In [142]:
import glob

for files in glob.glob("variable/Global_variable_per-country/who-immunization-coverage/*.csv"):
    data_immuno = pd.read_csv(files)
    print(files)
    data_immuno = data_immuno[[' 2018','Country']]
    df = df.merge(data_immuno, how='left', left_on='Territory', right_on='Country')
    name = files[39:43]
    print(name)
    df[name]= df[' 2018']
    df.drop([' 2018','Country'], axis=1, inplace=True)
    


variable/Global_variable_per-country/who-immunization-coverage/ROTAC.csv
o-im
variable/Global_variable_per-country/who-immunization-coverage/HepB3.csv
o-im
variable/Global_variable_per-country/who-immunization-coverage/DTP3.csv
o-im
variable/Global_variable_per-country/who-immunization-coverage/MCV2.csv
o-im
variable/Global_variable_per-country/who-immunization-coverage/MCV1.csv
o-im
variable/Global_variable_per-country/who-immunization-coverage/Hib3.csv
o-im
variable/Global_variable_per-country/who-immunization-coverage/PAB.csv
o-im
variable/Global_variable_per-country/who-immunization-coverage/Pol3.csv
o-im
variable/Global_variable_per-country/who-immunization-coverage/PCV3.csv
o-im
variable/Global_variable_per-country/who-immunization-coverage/BCG.csv
o-im


In [143]:
df.columns

Index(['Province/State', 'Country/Region', 'Territory', 'Population', 'Lat',
       'Long', '1/22/20', '1/23/20', '1/24/20', '1/25/20',
       ...
       'Corporate Tax Rate (%)', 'Gov't Expenditure % of GDP ',
       'GDP (Billions, PPP)', 'GDP Growth Rate (%)',
       '5 Year GDP Growth Rate (%)', 'GDP per Capita (PPP)',
       'Unemployment (%)', 'Inflation (%)', 'Public Debt (% of GDP)', 'o-im'],
      dtype='object', length=129)

## Split by continent

In [144]:
continent = pd.read_csv('variable/Global_variable_per-country/locations.csv')
continent.drop(['location', 'population_year','population'], axis=1, inplace=True)
df = df.merge(continent, how='left', left_on='Territory', right_on='countriesAndTerritories')
df.drop(['countriesAndTerritories'], axis=1, inplace=True)

In [145]:
index = df[df['continent'].isnull()].index
variable = 'continent'
df.at[index[0], variable] = 'Island'
df.at[index[1], variable] = 'Island'
df.at[index[2], variable] = 'South America'
df.at[index[3], variable] = 'Europe'
df.at[index[4], variable] = 'Asia'
df.at[index[5], variable] = 'Africa'
df.at[index[6], variable] = 'Island'
df.at[index[7], variable] = 'Africa'
df.at[index[8], variable] = 'Africa'
df.at[index[9],variable] = 'Africa'
df.at[index[10],variable] = 'Africa'
df.at[index[11], variable] = 'South America'
df.at[index[12], variable] = 'Africa'
df.at[index[13], variable] = 'Island'
df.at[index[14],variable] = 'Island'
df.at[index[15],variable] = 'South America'
df.at[index[16],variable] = 'Africa'
df.at[index[17], variable] = 'South America'
df.at[index[18], variable] = 'Island'
df.at[index[19], variable] = 'Island'
df.at[index[20], variable] = 'Island'
df.at[index[21], variable] = 'Island'
df.at[index[22],variable] = 'Island'
df.at[index[23],variable] = 'Island'
df.at[index[24],variable] = 'Island'
df.at[index[25],variable] = 'Africa'
df.at[index[26],variable] = 'Asia'
df.at[index[27],variable] = 'Asia'
df.at[index[28],variable] = 'Europe'
df.at[index[29],variable] = 'Island'
df.at[index[30],variable] = 'Island'
df.at[index[31],variable] = 'Island'
df.at[index[32], variable] = 'Africa'
df.at[index[33],variable] = 'Europe'
df.at[index[34],variable] = 'Island'
df.at[index[35], variable] = 'Asia'
df.at[index[36], variable] = 'Asia'
df.at[index[37], variable] = 'Island'
df.at[index[38], variable] = 'Island'
df.at[index[39],variable] = 'Europe'
df.at[index[40], variable] = 'Asia'
df.at[index[41], variable] = 'Africa'
df.at[index[42], variable] = 'Asia'
df.at[index[43], variable] = 'Africa'
df.at[index[44], variable] = 'Africa'
df.at[index[45], variable] = 'South America'
df.at[index[46], variable] = 'Asia'
df.at[index[47], variable] = 'Island'
df.at[index[48], variable] = 'Island'
df.at[index[49],variable] = 'Europe'
df.at[index[50], variable] = 'Island'
df.at[index[51],variable] = 'Europe'
df.at[index[52], variable] = 'North America'
df.at[index[53], variable] = 'Asia'
df.at[index[54], variable] = 'Asia'
df.at[index[55], variable] = 'Island'
df.at[index[56], variable] = 'Asia'
df.at[index[57], variable] = 'Africa'
df.at[index[58], variable] = 'Island'
df.at[index[59], variable] = 'Island'
df.at[index[60], variable] = 'Island'
df.at[index[61], variable] = 'Island'
df.at[index[62], variable] = 'Africa'
df.at[index[63], variable] = 'Island'
df.at[index[64], variable] = 'Island'
df.at[index[65], variable] = 'Africa'
df.at[index[66], variable] = 'Island'
df.at[index[67], variable] = 'Island'
df.at[index[68], variable] = 'Island'
df.at[index[69], variable] = 'Island'
df.at[index[70], variable] = 'Africa'
df.at[index[71], variable] = 'Island'
df.at[index[72], variable] = 'Island'
df.at[index[73], variable] = 'Island'
df.at[index[74], variable] = 'Island'
df.at[index[75], variable] = 'Asia'
df.at[index[76], variable] = 'Island'
df.at[index[77], variable] = 'Island'
df.at[index[78], variable] = 'Africa'
df.at[index[79],variable] = 'Asia'
df.at[index[80], variable] = 'Island'
df.at[index[81],variable] = 'Asia'
df.at[index[82], variable] = 'Island'
df.at[index[83], variable] = 'Island'
df.at[index[84],variable] = 'Asia'

In [146]:
df.continent.unique()

array(['Asia', 'Europe', 'Africa', 'Island', 'South America', 'Oceania',
       'North America'], dtype=object)

## Commorbidity / Risk Factors

### Obesity 

https://www.kaggle.com/arttua/who-obesity-by-country-2016

In [147]:
obesity = pd.read_csv('variable/Global_variable_per-country/obesity.csv')
obesity['Country'] = obesity['Unnamed: 0']
obesity.drop('Unnamed: 0', axis=1, inplace=True)

In [148]:
obesity.rename(columns={'Both.sexes':'pop_obsesity',
                        'Male':'Male_obesity','Female':'Female_obesity'}, inplace=True)

In [149]:
df = df.merge(obesity, how='left', left_on='Country/Region', right_on='Country')
df.drop(['Country'], axis=1, inplace=True)

## Medical care

### Hospital beds

https://www.kaggle.com/hamzael1/hospital-beds-by-country

In [150]:
beds = pd.read_csv('variable/Global_variable_per-country/beds.csv')

In [151]:
beds= beds[['Country Code', '2011']]

In [152]:
beds.rename(columns={'2011':'nb_beds'}, inplace=True)

In [153]:
df = df.merge(beds, how='left', left_on='Alpha-3 code', right_on='Country Code')
df.drop(['Country Code'], axis=1, inplace=True)

In [155]:
df.shape

(274, 134)

### Health Systems - WHO 


In [171]:
hs = pd.read_csv('variable/Global_variable_per-country/Health_systems.csv')
hs.drop(['Health_exp_pct_GDP_2016','Health_exp_public_pct_2016','Health_exp_out_of_pocket_pct_2016','Health_exp_per_capita_USD_2016','per_capita_exp_PPP_2016','External_health_exp_pct_2016','Completeness_of_birth_reg_2009-18','Completeness_of_death_reg_2008-16'], axis=1, inplace=True)

In [172]:
hs.drop(['Province_State','World_Bank_Name'], axis=1, inplace=True)

In [173]:
hs

,Country_Region,Physicians_per_1000_2009-18,Nurse_midwife_per_1000_2009-18,Specialist_surgical_per_1000_2008-18
0,Afghanistan,0.3,0.3,0.0
1,Albania,1.2,3.6,11.6
2,Algeria,1.8,2.2,12.1
3,Andorra,3.3,4.0,83.1
4,Angola,0.2,1.3,NaN
5,Antigua and Barbuda,2.8,3.1,14.0
6,Argentina,4.0,2.6,50.1
7,Armenia,2.9,5.6,86.7
8,Australia,3.6,12.7,45.1
9,Austria,5.1,8.2,109.9


### Pollution ?
https://www.kaggle.com/kweinmeister/pm25-global-air-pollution-20102017

In [107]:
df.shape

(420, 137)

In [179]:
df.shape

(274, 134)

In [73]:
df.to_csv('Global_variable_per_country.csv',index=False)